#AutoGluon Tabular

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#df = pd.read_csv("../data/cleaned_choreographies.csv")
df = pd.read_csv("/content/cleaned_choreographies.csv")
feature_columns = ["timeDuration",
"nMovements",
"movementsDifficulty",
"robotSpeech",
"acrobaticMovements",
"movementsRepetition",
"movementsTransitionsDuration",
"humanMovements",
"balance",
"speed",
"bodyPartsCombination",
"musicBPM",
"sameStartEndPositionPlace",
"headMovement",
"armsMovement",
"handsMovement",
"legsMovement",
"feetMovement",
"musicGenre_electronic",
"musicGenre_folk",
"musicGenre_indie",
"musicGenre_latin",
"musicGenre_pop",
"musicGenre_rap",
"musicGenre_rock"
]
target_columns = [c for c in df.columns if c not in feature_columns]
df = df[feature_columns + target_columns]


In [ ]:
df_binary = df.copy()
df_binary[target_columns] = (df_binary[target_columns] >= 4).astype(int)


In [ ]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 146.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
print(target_columns)

['EvaluationChoreographyStoryTelling', 'EvaluationChoreographyRhythm', 'EvaluationChoreographyMovementTechnique', 'EvaluationChoreographyPublicInvolvement', 'EvaluationChoreographySpaceUse', 'EvaluationChoreographyHumanCharacterization', 'EvaluationChoreographyHumanReproducibility']


## StoryTelling

In [ ]:
X = df_binary[feature_columns].copy()

In [ ]:

target_name = target_columns[0]
print(f"==========********{target_name}********==========")
y = df_binary[target_name].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)
train_data.to_csv(f'train_{target_name}.csv', index=False)
test_data.to_csv(f'test_{target_name}.csv', index=False)
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)
label = target_name
predictor = TabularPredictor(label=label, eval_metric = "f1").fit(train_data)
y_pred = predictor.predict(test_data.drop(columns=[label]))


No path specified. Models will be saved in: "AutogluonModels/ag-20250605_131732"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.61 GB / 12.67 GB (83.7%)
Disk Space Avail:   69.81 GB / 112.64 GB (62.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competition

==========EvaluationChoreographyStoryTelling==========


User-specified model hyperparameters to be fit:
{
	'NN_TORCH': [{}],
	'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, {'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'ag_args': {'name_suffix': 'Large', 'priority': 0, 'hyperparameter_tune_kwargs': None}}],
	'CAT': [{}],
	'XGB': [{}],
	'FASTAI': [{}],
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regressi

In [ ]:
predictor.evaluate(test_data, silent=True)


{'f1': 0.5762463343108505,
 'accuracy': 0.6625802685347344,
 'balanced_accuracy': np.float64(0.667139099763224),
 'mcc': np.float64(0.3170240917996556),
 'roc_auc': np.float64(0.7180794908096761),
 'precision': 0.4993646759847522,
 'recall': 0.6811091854419411}

In [ ]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.576246,0.553648,f1,0.116772,0.049289,15.919759,0.002830,0.002046,0.342695,2,True,14
1,LightGBM,0.576014,0.496386,f1,0.008528,0.002746,1.327806,0.008528,0.002746,1.327806,1,True,4
2,RandomForestEntr,0.571181,0.436548,f1,0.170255,0.084340,2.022742,0.170255,0.084340,2.022742,1,True,6
3,LightGBMXT,0.569957,0.445570,f1,0.011689,0.003925,1.422719,0.011689,0.003925,1.422719,1,True,3
4,LightGBMLarge,0.565878,0.463054,f1,0.032969,0.006316,2.111669,0.032969,0.006316,2.111669,1,True,13
5,RandomForestGini,0.565068,0.432292,f1,0.186665,0.084132,1.869236,0.186665,0.084132,1.869236,1,True,5
6,XGBoost,0.562660,0.490291,f1,0.015610,0.003705,1.280916,0.015610,0.003705,1.280916,1,True,11
7,CatBoost,0.559930,0.457002,f1,0.008974,0.001076,0.833306,0.008974,0.001076,0.833306,1,True,7
8,NeuralNetFastAI,0.555138,0.523013,f1,0.028101,0.011092,3.714222,0.028101,0.011092,3.714222,1,True,10
9,NeuralNetTorch,0.552553,0.496842,f1,0.023898,0.012301,10.520424,0.023898,0.012301,10.520424,1,True,12


##  Rhythm

In [ ]:

target_name = target_columns[1]
print(f"==========********{target_name}********==========")
y = df_binary[target_name].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)
train_data.to_csv(f'train_{target_name}.csv', index=False)
test_data.to_csv(f'test_{target_name}.csv', index=False)
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)
label = target_name
predictor = TabularPredictor(label=label, eval_metric = "f1").fit(train_data)
y_pred = predictor.predict(test_data.drop(columns=[label]))


No path specified. Models will be saved in: "AutogluonModels/ag-20250605_132608"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.36 GB / 12.67 GB (81.7%)
Disk Space Avail:   69.55 GB / 112.64 GB (61.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competition

==========********EvaluationChoreographyRhythm********==========


	Types of features in original data (raw dtype, special dtypes):
		('int', []) : 25 | ['timeDuration', 'nMovements', 'movementsDifficulty', 'robotSpeech', 'acrobaticMovements', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('int', [])       : 16 | ['timeDuration', 'nMovements', 'movementsDifficulty', 'acrobaticMovements', 'movementsRepetition', ...]
		('int', ['bool']) :  9 | ['robotSpeech', 'sameStartEndPositionPlace', 'musicGenre_electronic', 'musicGenre_folk', 'musicGenre_indie', ...]
	0.1s = Fit runtime
	25 features in original data used to generate 25 features in processed data.
	Train Data (Processed) Memory Usage: 0.90 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.14s ...
AutoGluon will gauge predictive performance using evaluation metric: 'f1'
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.1, Train Rows: 6165, Val Rows: 685
Use

In [ ]:
predictor.evaluate(test_data, silent=True)


{'f1': 0.6278424847476428,
 'accuracy': 0.6082895504962055,
 'balanced_accuracy': np.float64(0.6309419143074849),
 'mcc': np.float64(0.26785463056392583),
 'roc_auc': np.float64(0.7276913930685691),
 'precision': 0.5255338904363974,
 'recall': 0.7796143250688705}

In [ ]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,0.646048,0.596154,f1,0.013850,0.003308,1.601030,0.013850,0.003308,1.601030,1,True,13
1,LightGBM,0.643626,0.605010,f1,0.004463,0.001776,1.178340,0.004463,0.001776,1.178340,1,True,4
2,CatBoost,0.642733,0.597701,f1,0.004379,0.001137,0.951500,0.004379,0.001137,0.951500,1,True,7
3,XGBoost,0.641531,0.614504,f1,0.010698,0.003494,1.169138,0.010698,0.003494,1.169138,1,True,11
4,RandomForestGini,0.638412,0.585742,f1,0.160728,0.084262,1.928872,0.160728,0.084262,1.928872,1,True,5
5,RandomForestEntr,0.633133,0.595104,f1,0.158604,0.104975,2.094223,0.158604,0.104975,2.094223,1,True,6
6,LightGBMXT,0.632005,0.582418,f1,0.019182,0.006225,1.868290,0.019182,0.006225,1.868290,1,True,3
7,WeightedEnsemble_L2,0.627842,0.622642,f1,0.458975,0.249972,22.380155,0.003735,0.002509,0.347520,2,True,14
8,ExtraTreesEntr,0.627528,0.533333,f1,0.189953,0.141576,2.527464,0.189953,0.141576,2.527464,1,True,9
9,NeuralNetTorch,0.619256,0.611807,f1,0.016471,0.008028,9.736349,0.016471,0.008028,9.736349,1,True,12


## MovementTechnique

In [ ]:

target_name = target_columns[2]
print(f"==========********{target_name}********==========")
y = df_binary[target_name].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)
train_data.to_csv(f'train_{target_name}.csv', index=False)
test_data.to_csv(f'test_{target_name}.csv', index=False)
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)
label = target_name
predictor = TabularPredictor(label=label, eval_metric = "f1").fit(train_data)
y_pred = predictor.predict(test_data.drop(columns=[label]))


No path specified. Models will be saved in: "AutogluonModels/ag-20250605_132643"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.35 GB / 12.67 GB (81.7%)
Disk Space Avail:   69.26 GB / 112.64 GB (61.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competition

==========********EvaluationChoreographyMovementTechnique********==========


	0.401	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.401	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.4495	 = Validation score   (f1)
	2.12s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM ...
	0.4532	 = Validation score   (f1)
	3.08s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.3896	 = Validation score   (f1)
	1.92s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.3907	 = Validation score   (f1)
	2.09s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ...
	0.4319	 = Validation score   (f1)
	0.88s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.2976	 = Validation score   (f1)
	1.63s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesEntr ...

In [ ]:
predictor.evaluate(test_data, silent=True)


{'f1': 0.5340974212034384,
 'accuracy': 0.5253940455341506,
 'balanced_accuracy': np.float64(0.5911780943251999),
 'mcc': np.float64(0.18496444445795637),
 'roc_auc': np.float64(0.6422051882499968),
 'precision': 0.40137812230835485,
 'recall': 0.797945205479452}

In [ ]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.540127,0.453202,f1,0.055236,0.022206,3.082523,0.055236,0.022206,3.082523,1,True,4
1,CatBoost,0.539942,0.431877,f1,0.004902,0.001078,0.880128,0.004902,0.001078,0.880128,1,True,7
2,LightGBMXT,0.537419,0.449495,f1,0.026165,0.021615,2.121879,0.026165,0.021615,2.121879,1,True,3
3,LightGBMLarge,0.537294,0.441748,f1,0.053163,0.023532,2.872031,0.053163,0.023532,2.872031,1,True,13
4,WeightedEnsemble_L2,0.534097,0.542574,f1,0.140435,0.057064,48.998191,0.002775,0.003257,0.486070,2,True,14
5,RandomForestEntr,0.533553,0.390746,f1,0.148347,0.084648,2.093570,0.148347,0.084648,2.093570,1,True,6
6,RandomForestGini,0.531098,0.389610,f1,0.160431,0.094667,1.915509,0.160431,0.094667,1.915509,1,True,5
7,XGBoost,0.525226,0.437956,f1,0.037140,0.013291,2.390460,0.037140,0.013291,2.390460,1,True,11
8,ExtraTreesGini,0.522558,0.297619,f1,0.169621,0.084718,1.626677,0.169621,0.084718,1.626677,1,True,8
9,KNeighborsUnif,0.519115,0.400966,f1,0.066109,0.021449,0.013832,0.066109,0.021449,0.013832,1,True,1


 ## PublicInvolvement

In [ ]:

target_name = target_columns[3]
print(f"==========********{target_name}********==========")
y = df_binary[target_name].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)
train_data.to_csv(f'train_{target_name}.csv', index=False)
test_data.to_csv(f'test_{target_name}.csv', index=False)
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)
label = target_name
predictor = TabularPredictor(label=label, eval_metric = "f1").fit(train_data)
y_pred = predictor.predict(test_data.drop(columns=[label]))


No path specified. Models will be saved in: "AutogluonModels/ag-20250605_132749"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.41 GB / 12.67 GB (82.2%)
Disk Space Avail:   68.99 GB / 112.64 GB (61.2%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competition

==========********EvaluationChoreographyPublicInvolvement********==========


	0.3956	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.3945	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.4088	 = Validation score   (f1)
	3.03s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBM ...
	0.4386	 = Validation score   (f1)
	1.47s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.3416	 = Validation score   (f1)
	1.91s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.3406	 = Validation score   (f1)
	2.2s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: CatBoost ...
	0.3735	 = Validation score   (f1)
	1.65s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.1832	 = Validation score   (f1)
	1.59s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesEntr ...

In [ ]:
predictor.evaluate(test_data, silent=True)


{'f1': 0.48674959437533805,
 'accuracy': 0.4460011675423234,
 'balanced_accuracy': np.float64(0.5586258151131569),
 'mcc': np.float64(0.12888446817841595),
 'roc_auc': np.float64(0.6399701125175808),
 'precision': 0.34065102195306585,
 'recall': 0.8522727272727273}

In [ ]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsUnif,0.507479,0.395604,f1,0.049852,0.021115,0.014264,0.049852,0.021115,0.014264,1,True,1
1,KNeighborsDist,0.507479,0.394521,f1,0.051522,0.022093,0.012708,0.051522,0.022093,0.012708,1,True,2
2,LightGBMXT,0.499735,0.408840,f1,0.090605,0.027303,3.028531,0.090605,0.027303,3.028531,1,True,3
3,LightGBMLarge,0.498660,0.420455,f1,0.043260,0.009289,2.304251,0.043260,0.009289,2.304251,1,True,13
4,XGBoost,0.495197,0.448087,f1,0.022629,0.021983,2.069323,0.022629,0.021983,2.069323,1,True,11
5,LightGBM,0.492768,0.438596,f1,0.014344,0.005052,1.471908,0.014344,0.005052,1.471908,1,True,4
6,WeightedEnsemble_L2,0.486750,0.482587,f1,0.054736,0.037505,22.700222,0.002412,0.002181,0.339141,2,True,14
7,RandomForestEntr,0.485483,0.340557,f1,0.147976,0.124026,2.201284,0.147976,0.124026,2.201284,1,True,6
8,CatBoost,0.483151,0.373494,f1,0.004703,0.002644,1.653131,0.004703,0.002644,1.653131,1,True,7
9,NeuralNetFastAI,0.482611,0.436548,f1,0.028519,0.010074,5.253879,0.028519,0.010074,5.253879,1,True,10


## SpaceUse

In [ ]:

target_name = target_columns[4]
print(f"==========********{target_name}********==========")
y = df_binary[target_name].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)
train_data.to_csv(f'train_{target_name}.csv', index=False)
test_data.to_csv(f'test_{target_name}.csv', index=False)
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)
label = target_name
predictor = TabularPredictor(label=label, eval_metric = "f1").fit(train_data)
y_pred = predictor.predict(test_data.drop(columns=[label]))


No path specified. Models will be saved in: "AutogluonModels/ag-20250605_132834"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.41 GB / 12.67 GB (82.2%)
Disk Space Avail:   68.72 GB / 112.64 GB (61.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competition

==========********EvaluationChoreographySpaceUse********==========


	0.3127	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.3127	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.3309	 = Validation score   (f1)
	3.72s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: LightGBM ...
	0.3206	 = Validation score   (f1)
	1.8s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.2305	 = Validation score   (f1)
	1.78s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.2531	 = Validation score   (f1)
	1.92s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ...
	0.1867	 = Validation score   (f1)
	0.87s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.0952	 = Validation score   (f1)
	1.46s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: ExtraTreesEntr ..

In [ ]:
predictor.evaluate(test_data, silent=True)


{'f1': 0.4469820554649266,
 'accuracy': 0.6042031523642732,
 'balanced_accuracy': np.float64(0.6151751889648172),
 'mcc': np.float64(0.20026049411768077),
 'roc_auc': np.float64(0.6436277619677717),
 'precision': 0.3442211055276382,
 'recall': 0.6372093023255814}

In [ ]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetFastAI,0.446982,0.453865,f1,0.028831,0.009814,3.649994,0.028831,0.009814,3.649994,1,True,10
1,WeightedEnsemble_L2,0.446982,0.453865,f1,0.033055,0.011910,3.991252,0.004223,0.002095,0.341259,2,True,14
2,NeuralNetTorch,0.362768,0.423881,f1,0.017284,0.008712,16.179504,0.017284,0.008712,16.179504,1,True,12
3,LightGBM,0.343575,0.320611,f1,0.025877,0.007679,1.799419,0.025877,0.007679,1.799419,1,True,4
4,LightGBMLarge,0.336662,0.348148,f1,0.072813,0.016283,3.495152,0.072813,0.016283,3.495152,1,True,13
5,XGBoost,0.335645,0.334545,f1,0.015993,0.004924,1.400845,0.015993,0.004924,1.400845,1,True,11
6,KNeighborsUnif,0.328276,0.312704,f1,0.051298,0.021905,0.014303,0.051298,0.021905,0.014303,1,True,1
7,LightGBMXT,0.328234,0.330935,f1,0.083237,0.045557,3.718385,0.083237,0.045557,3.718385,1,True,3
8,KNeighborsDist,0.326475,0.312704,f1,0.051511,0.020626,0.012130,0.051511,0.020626,0.012130,1,True,2
9,RandomForestEntr,0.311178,0.253061,f1,0.163671,0.084321,1.917858,0.163671,0.084321,1.917858,1,True,6


## HumanCharacterization

In [ ]:

target_name = target_columns[5]
print(f"==========********{target_name}********==========")
y = df_binary[target_name].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)
train_data.to_csv(f'train_{target_name}.csv', index=False)
test_data.to_csv(f'test_{target_name}.csv', index=False)
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)
label = target_name
predictor = TabularPredictor(label=label, eval_metric = "f1").fit(train_data)
y_pred = predictor.predict(test_data.drop(columns=[label]))


No path specified. Models will be saved in: "AutogluonModels/ag-20250605_132916"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.41 GB / 12.67 GB (82.2%)
Disk Space Avail:   68.47 GB / 112.64 GB (60.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competition

==========********EvaluationChoreographyHumanCharacterization********==========


	0.4016	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.4016	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBMXT ...


[1000]	valid_set's binary_logloss: 0.716001	valid_set's f1: 0.366738


	0.3932	 = Validation score   (f1)
	3.09s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBM ...
	0.4376	 = Validation score   (f1)
	1.73s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.3147	 = Validation score   (f1)
	2.01s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.3202	 = Validation score   (f1)
	3.06s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ...
	0.3	 = Validation score   (f1)
	1.07s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.2128	 = Validation score   (f1)
	1.62s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.2263	 = Validation score   (f1)
	1.67s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.4486	 = Validation score   (f1)
	5.7s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	0.

In [ ]:
predictor.evaluate(test_data, silent=True)


{'f1': 0.5725288831835686,
 'accuracy': 0.4168126094570928,
 'balanced_accuracy': np.float64(0.5109009769321262),
 'mcc': np.float64(0.05807899631480913),
 'roc_auc': np.float64(0.5385856932580886),
 'precision': 0.4047186932849365,
 'recall': 0.9780701754385965}

In [ ]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,0.575731,0.468619,f1,0.088238,0.020046,3.018392,0.088238,0.020046,3.018392,1,True,13
1,LightGBM,0.572894,0.437637,f1,0.024252,0.007315,1.726643,0.024252,0.007315,1.726643,1,True,4
2,WeightedEnsemble_L2,0.572529,0.493739,f1,0.075792,0.034388,22.858891,0.003090,0.002107,0.332318,2,True,14
3,LightGBMXT,0.571063,0.393162,f1,0.094446,0.030246,3.093446,0.094446,0.030246,3.093446,1,True,3
4,XGBoost,0.571053,0.458418,f1,0.028785,0.009208,2.150316,0.028785,0.009208,2.150316,1,True,11
5,ExtraTreesGini,0.570832,0.212766,f1,0.172835,0.085082,1.615525,0.172835,0.085082,1.615525,1,True,8
6,CatBoost,0.570711,0.300000,f1,0.004850,0.001212,1.069906,0.004850,0.001212,1.069906,1,True,7
7,RandomForestGini,0.570711,0.314721,f1,0.163713,0.140732,2.008037,0.163713,0.140732,2.008037,1,True,5
8,ExtraTreesEntr,0.570593,0.226316,f1,0.208873,0.084191,1.669191,0.208873,0.084191,1.669191,1,True,9
9,RandomForestEntr,0.569636,0.320197,f1,0.161748,0.084245,3.059149,0.161748,0.084245,3.059149,1,True,6


## HumanReproducibility

In [ ]:

target_name = target_columns[6]
print(f"==========********{target_name}********==========")
y = df_binary[target_name].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)
train_data.to_csv(f'train_{target_name}.csv', index=False)
test_data.to_csv(f'test_{target_name}.csv', index=False)
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)
label = target_name
predictor = TabularPredictor(label=label, eval_metric = "f1").fit(train_data)
y_pred = predictor.predict(test_data.drop(columns=[label]))


No path specified. Models will be saved in: "AutogluonModels/ag-20250605_132959"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.37 GB / 12.67 GB (81.8%)
Disk Space Avail:   68.16 GB / 112.64 GB (60.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competition

==========********EvaluationChoreographyHumanReproducibility********==========


	0.838	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.838	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.8694	 = Validation score   (f1)
	1.76s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	0.8701	 = Validation score   (f1)
	1.43s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.8675	 = Validation score   (f1)
	1.85s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8692	 = Validation score   (f1)
	2.01s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ...
	0.8678	 = Validation score   (f1)
	0.99s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.8678	 = Validation score   (f1)
	1.52s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesEntr ...


In [ ]:
predictor.evaluate(test_data, silent=True)


{'f1': 0.8643449419568823,
 'accuracy': 0.7612375948628137,
 'balanced_accuracy': np.float64(0.500079956052391),
 'mcc': np.float64(0.0014100973798310131),
 'roc_auc': np.float64(0.579567560042292),
 'precision': 0.7624341720304272,
 'recall': 0.9977029096477795}

In [ ]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.865187,0.867769,f1,0.006418,0.001158,0.994507,0.006418,0.001158,0.994507,1,True,7
1,XGBoost,0.865187,0.868704,f1,0.010108,0.002948,1.093981,0.010108,0.002948,1.093981,1,True,11
2,LightGBMXT,0.864811,0.869421,f1,0.029693,0.010018,1.762039,0.029693,0.010018,1.762039,1,True,3
3,NeuralNetTorch,0.864721,0.868704,f1,0.015600,0.009379,4.770519,0.015600,0.009379,4.770519,1,True,12
4,ExtraTreesEntr,0.864721,0.868704,f1,0.233691,0.083989,1.519272,0.233691,0.083989,1.519272,1,True,9
5,ExtraTreesGini,0.864721,0.867769,f1,0.271252,0.084247,1.516390,0.271252,0.084247,1.516390,1,True,8
6,LightGBMLarge,0.864345,0.870141,f1,0.008976,0.013411,1.539607,0.008976,0.013411,1.539607,1,True,13
7,LightGBM,0.864345,0.870141,f1,0.033846,0.002465,1.432038,0.033846,0.002465,1.432038,1,True,4
8,WeightedEnsemble_L2,0.864345,0.870141,f1,0.035338,0.005397,1.898655,0.001492,0.002933,0.466618,2,True,14
9,RandomForestEntr,0.863968,0.869205,f1,0.240516,0.083966,2.005316,0.240516,0.083966,2.005316,1,True,6
